In [31]:
using Revise
using Test

In [64]:
using LinearAlgebra
using ITensors
import CTMRG

In [33]:
#３本足テンソルを作るコード
function P_edge_end(β, J, σind1, σind2, sind1)
    dumyind = Index(2, "dummy")
    T = CTMRG.ising_transfer_tensor_2D(β, J, σind1, σind2, sind1, dumyind)
    D = CTMRG.fixend(dumyind)
    P = T*D
    return P
end
function P_centor(β, J, σind1, sind1, sind2)
    dumyind = Index(2, "dummy")
    T = CTMRG.ising_transfer_tensor_2D(β, J, σind1, dumyind, sind1, sind2)
    D = CTMRG.fixend(dumyind)
    P = T*D
    return P
end
#２本足のテンソルを作るコード
function C_corner(β, J, σind1, sind1)
    dumyind_σ = Index(2, "dummy_σ")
    dumyind_s = Index(2, "dummy_s")
    T = CTMRG.ising_transfer_tensor_2D(β, J, σind1, dumyind_σ, sind1, dumyind_s)
    D_σ = CTMRG.fixend(dumyind_σ)
    D_s = CTMRG.fixend(dumyind_s)
    C = T*D_σ*D_s
    return C
end

C_corner (generic function with 1 method)

In [34]:
sind11 = Index(2, "s,i=1,j=1")
sind12 = Index(2, "s,i=1,j=2")
sind21 = Index(2, "s,i=2,j=1")
sind22 = Index(2, "s,i=2,j=2")
σind11 = Index(2, "σ,i=1,j=1")
σind12 = Index(2, "σ,i=1,j=2")
σind21 = Index(2, "σ,i=2,j=1")
σind22 = Index(2, "σ,i=2,j=2")

(dim=2|id=265|"i=2,j=2,σ")

In [61]:
cinds = combiner(sind12, sind22, tags="s_all")
cindσ = combiner(σind21, σind22, tags="σ_all")
@show inds(cinds)[1]
@show inds(cindσ)[1]
;

(inds(cinds))[1] = (dim=4|id=815|"s_all")
(inds(cindσ))[1] = (dim=4|id=473|"σ_all")


$$ 拡大

In [ ]:
#拡大の練習
β = 1.0
J = 1.0
P_edge = P_edge_end(β, J, σind11, σind21, sind21)
P_c = P_centor(β, J, σind12, sind11, sind12)
C = C_corner(β, J, σind11, sind11)
W = CTMRG.ising_transfer_tensor_2D(β, J, σind12, σind22, sind21, sind22)
_P_edge = P_edge * W
C_expand = P_c * C * _P_edge
@show _C_expand = C_expand * cindσ * cinds
;


_C_expand = C_expand * cindσ * cinds = ITensor ord=2
Dim 1: (dim=4|id=753|"σ_all")
Dim 2: (dim=4|id=497|"s_all")
NDTensors.Dense{Float64, Vector{Float64}}
 4×4
      8.898259745056167e6  166015.77678748913    166015.77678748913   6079.149576352778
 166015.7767874891           3151.8440153356046    3151.844015335604   168.8870346817601
 166015.7767874891           3151.8440153356046  172084.85243036106   9220.919658207573
   6079.149576352777          168.8870346817601    9220.919658207571  3310.6571165365535


$$ 圧縮

In [62]:
C_array = array(_C_expand)
evals, evecs = eigen(C_array)
#ievecs = inv(evecs)
Λ, A, iA = CTMRG.make_diag_Tesnor(evals, evecs, inds(cindσ)[1], inds(cinds)[1])
;

In [ ]:
function make_CTMRG_ind_stags(L::Int)
    i = 2 * L # 行
    j = 2 * L -1 # 列
    sind_list = Array{Index}(undef, i, j)
    for m in 1:j, n in 1:i
        sind_list[n, m] = Index(2, "s, i=$n, j=$m")
    end
    return sind_list
end
function make_CTMRG_ind_σtags(L::Int)
    i = 2 * L - 1 #　行
    j = 2 * L # 列
    σind_list = Array{Index}(undef, i, j)
    for m in 1:j, n in 1:i
        σind_list[n, m] = Index(2, "σ, i=$n, j=$m")
    end
    return σind_list
end

In [ ]:
L = 1
β = 1.0
J = 1.0
T_list, T1_list = CTMRG.contruct_with_boundary(L, β, J)
@show inds(T1_list[1, 1])
@show inds(T1_list[1, 2])
@show inds(T1_list[2,1])
@show inds(T1_list[2,2])
;

inds(T1_list[1, 1]) = ((dim=2|id=768|"i=1,j=2,σ"), (dim=2|id=843|"i=2,j=1,s"))
inds(T1_list[1, 2]) = ((dim=2|id=879|"i=2,j=1,σ"), (dim=2|id=796|"i=2,j=2,σ"), (dim=2|id=843|"i=2,j=1,s"), (dim=2|id=874|"i=3,j=1,s"), (dim=2|id=870|"i=1,j=2,s"), (dim=2|id=470|"i=1,j=3,σ"))
inds(T1_list[2, 1]) = ((dim=2|id=768|"i=1,j=2,σ"), (dim=2|id=470|"i=1,j=3,σ"), (dim=2|id=870|"i=1,j=2,s"), (dim=2|id=779|"i=2,j=2,s"), (dim=2|id=879|"i=2,j=1,σ"), (dim=2|id=874|"i=3,j=1,s"))
inds(T1_list[2, 2]) = ((dim=2|id=796|"i=2,j=2,σ"), (dim=2|id=779|"i=2,j=2,s"))


In [ ]:
C_down_left = T1_list[1, 1]
C_down_right = T1_list[2, 1]
C_up_left = T1_list[1, 2]
C_up_right = T1_list[2, 2]

ITensor ord=2 (dim=2|id=796|"i=2,j=2,σ") (dim=2|id=779|"i=2,j=2,s")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
#Pを定義する関数/Wを定義する関数